In [18]:
import numpy as np
import pandas as pd

# Gradient Descent

In [3]:
def gd(theta, grad, eta, data, max_iter=10):
    # Takes in original parameter estimate theta, gradient function grad, step-size eta
    # Returns parameter estimates obtained after max_iter steps
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta + eta * grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# Newton-Raphson

In [4]:
def nr(theta, grad, data, inverse_hessian, max_iter=10, eta = 1):
    # Takes in original parameter estimate theta, gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta - eta*inverse_hessian(theta, data) @ grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# Fisher's scoring

In [5]:
def fisher(theta, grad, data, inverse_info, max_iter=10, eta = 1):
    # Takes in original parameter estimate theta, gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta - eta*inverse_info(theta, data) @ grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# BFGS

In [17]:
def bfgs(f, theta, grad, data, max_iter=10):
    # Takes in original parameter estimate theta, negative likelihood function f, negative gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    I = np.diag(np.ones(theta.shape[0]))
    B = np.copy(I)
    B_inverse = np.copy(I)
    for i in range(max_iter):
        p = -B_inverse @ grad(theta, data)
        alpha = line_search(f, grad, p , theta, 1, data)
        s = alpha * p
        next_theta = (theta + s)
        y = grad(next_theta, data) - grad(theta, data)
        B = B + (y @ y.T)/(y.T @ s) - (B @ s @ s.T @ B.T)/(s.T @ B @ s)
        B_inverse = (I - (s @ y.T)/(y.T @ s)) @ B_inverse @ (I - (s @ y.T)/(y.T @ s)) + (s @ s.T)/(y.T @ s)
        theta = next_theta
        thetas[:, i+1] = theta.flatten()
    return thetas

def line_search(f, grad, p, theta, alpha, data, lower = 0, upper = float("inf"), c1 = 0.0001, c2 = 0.1):
    if f(theta, data) + c1 * alpha * p.T @ grad(theta, data) - f(theta + alpha * p, data) < 0:
        return line_search(f, grad, p, theta, 0.5*(lower + alpha), data, lower = 0, upper = alpha)
    elif -c2 * p.T @ grad(theta, data) + p.T @ grad(theta + alpha * p, data) < 0:
        return line_search(f, grad, p, theta, 2*alpha, data, lower = alpha, upper = float("inf"))
    return alpha

# Example 1 - Poisson, 1 Dimension

In [84]:
np.random.seed(0)
data = np.random.poisson(lam = 56.3, size = 1000)

In [85]:
def l1(theta, data):
    return np.sum(data)*np.log(theta) - len(data)*theta

def neg_l1(theta, data):
    return -l1(theta, data)
    
def grad1(theta, data):
    return np.sum(data)/theta - len(data)

def neg_grad1(theta, data):
    return -grad1(theta, data)

def inverse_hessian1(theta, data):
    return -(theta**2)/np.sum(data)

def inverse_info1(theta, data):
    return -(theta)/len(data)

In [86]:
theta0 = np.array([[5]])

In [87]:
%%timeit
gd(theta0, grad1, 1/len(data), data, max_iter = 100).flatten()

array([ 5.        , 15.2796    , 17.97066521, 20.10900201, 21.91361658,
       23.48726753, 24.88848342, 26.1545114 , 27.31085067, 28.37589071,
       29.36342303, 30.28411191, 31.14640855, 31.95714702, 32.72194786,
       33.44550051, 34.1317662 , 34.78412722, 35.40549894, 35.99841534,
       36.56509541, 37.10749533, 37.62734999, 38.12620654, 38.60545153,
       39.06633329, 39.50998043, 39.93741723, 40.34957665, 40.74731126,
       41.13140259, 41.50256905, 41.86147288, 42.208726  , 42.54489521,
       42.87050666, 43.18604977, 43.49198075, 43.78872555, 44.07668265,
       44.35622541, 44.6277042 , 44.89144835, 45.14776781, 45.3969547 ,
       45.63928473, 45.87501838, 46.10440209, 46.32766923, 46.54504107,
       46.75672761, 46.96292836, 47.16383303, 47.35962219, 47.55046784,
       47.73653398, 47.91797711, 48.09494666, 48.26758545, 48.43603006,
       48.6004112 , 48.76085405, 48.91747859, 49.07039983, 49.21972814,
       49.3655695 , 49.50802568, 49.64719451, 49.78317008, 49.91

In [88]:
%%timeit
gd(theta0, grad1, 10/len(data), data, max_iter = 50).flatten()

array([  5.        , 107.796     , 103.02791955,  98.50196972,
        94.2275405 ,  90.21283946,  86.46449946,  82.98717644,
        79.78316618,  76.85207592,  74.19058946,  71.79236219,
        69.6480725 ,  67.74564046,  66.07060419,  64.60662421,
        63.33606963,  62.24063228,  61.30191555,  60.50195433,
        59.82363668,  59.25101407,  58.76950095,  58.36597516,
        58.02879666,  57.74776436,  57.51402988,  57.31998507,
        57.15913627,  57.02597536,  56.91585443,  56.82486846,
        56.74974845,  56.68776607,  56.63664991,  56.5945129 ,
        56.55978994,  56.53118482,  56.50762529,  56.4882252 ,
        56.47225281,  56.45910426,  56.44828152,  56.43937398,
        56.43204329,  56.42601067,  56.42104653,  56.41696179,
        56.41360078,  56.41083535,  56.40856002])

In [89]:
%%timeit
nr(theta0, grad1, data, inverse_hessian1, max_iter=10).flatten()

array([ 5.        ,  9.55672187, 17.49404353, 29.56162586, 43.6282036 ,
       53.50662619, 56.24976704, 56.39761039, 56.398     , 56.398     ,
       56.398     ])

In [90]:
%%timeit
fisher(theta0, grad1, data, inverse_info1, max_iter=10).flatten()

array([ 5.   , 56.398, 56.398, 56.398, 56.398, 56.398, 56.398, 56.398,
       56.398, 56.398, 56.398])

In [91]:
%%timeit
bfgs(neg_l1, theta0, neg_grad1, data).flatten()

array([  5.        , 165.61875   ,  10.69020643, 144.91602358,
        44.24479993,  59.85877053,  54.42875174,  56.51883968,
        56.40221936,  56.39799096,  56.398     ])

# Example 2 - Normal Distribution, 2 Dimensions

In [141]:
np.random.seed(0)
data = np.random.normal(50, 5, size = 1000)

In [210]:
def l2(theta, data):
    return -(1/2)*np.sum(((data - theta[0])/theta[1])**2) - len(data)*np.log(theta[1])

def neg_l2(theta, data):
    return -l2(theta, data)
    
def grad2(theta, data):
    return np.array([(np.sum(data) - len(data)*theta[0])/theta[1]**2, 
                     np.sum((data - theta[0])**2/theta[1]**3) - len(data)/theta[1]])

def neg_grad2(theta, data):
    return -grad2(theta, data)

def inverse_hessian2(theta, data):
    return np.linalg.inv(np.r_[np.c_[-len(data)/theta[1]**2, 
                                     (-2*(np.sum(data) - len(data)*theta[0]))/theta[1]**3], 
                               np.c_[(-2*(np.sum(data) - len(data)*theta[0]))/theta[1]**3,
                                     -3*np.sum((data - theta[0])**2/theta[1]**4) + len(data)/theta[1]**2]])

def inverse_info2(theta, data):
    return np.linalg.inv(np.r_[np.c_[-len(data)/theta[1]**2, 
                                     0], 
                               np.c_[0,
                                     -2*len(data)/theta[1]]])

In [190]:
theta0 = np.array([10,2])[:, None]

In [177]:
#%%timeit
gd(theta0, grad2, 0.2/len(data), data, max_iter = 10000)

array([[10.        , 11.98868582, 11.99295811, ..., 49.77371493,
        49.77371494, 49.77371495],
       [ 2.        , 42.05760957, 42.05675794, ...,  4.93516579,
         4.93516579,  4.93516579]])

In [203]:
gd(np.array([44,2])[:, None], grad2, 1/len(data), data, max_iter = 100)

array([[44.        , 45.44342912, 45.50048954, 45.55744463, 45.61429336,
        45.67103454, 45.7276668 , 45.78418856, 45.84059807, 45.89689336,
        45.95307224, 46.00913231, 46.06507091, 46.12088517, 46.17657195,
        46.23212783, 46.28754915, 46.34283193, 46.39797192, 46.45296454,
        46.50780489, 46.56248774, 46.61700752, 46.6713583 , 46.72553375,
        46.7795272 , 46.83333155, 46.88693929, 46.94034251, 46.99353285,
        47.0465015 , 47.0992392 , 47.15173623, 47.2039824 , 47.25596702,
        47.30767893, 47.35910648, 47.41023751, 47.46105939, 47.51155899,
        47.56172269, 47.61153641, 47.6609856 , 47.71005526, 47.75872996,
        47.80699389, 47.85483084, 47.90222429, 47.94915739, 47.99561306,
        48.04157399, 48.08702273, 48.13194176, 48.17631349, 48.2201204 ,
        48.26334508, 48.30597032, 48.3479792 , 48.38935513, 48.43008199,
        48.4701442 , 48.50952681, 48.54821557, 48.58619704, 48.62345865,
        48.65998882, 48.69577698, 48.73081366, 48.7

In [192]:
#%%timeit
nr(theta0, grad2, data, inverse_hessian2, max_iter=10)

array([[ 1.00000000e+01, -3.23349838e+01, -1.15640238e+02,
        -2.81643301e+02, -6.13353761e+02, -1.27662782e+03,
        -2.60310263e+03, -5.25601562e+03, -1.05618233e+04,
        -2.11734294e+04, -4.23966371e+04],
       [ 2.00000000e+00,  4.06439598e+00,  8.15840678e+00,
         1.63313413e+01,  3.26699126e+01,  6.53434361e+01,
         1.30688677e+02,  2.61378257e+02,  5.22756964e+02,
         1.04551415e+03,  2.09102842e+03]])

In [204]:
nr(np.array([44,2])[:, None], grad2, data, inverse_hessian2, max_iter=10)

array([[44.        , 53.96844118, 50.57424055, 50.26822374, 50.06943558,
        49.93741953, 49.8514209 , 49.80086314, 49.77875387, 49.77396276,
        49.77371723],
       [ 2.        ,  1.27347927,  1.39499509,  1.82585984,  2.37179887,
         3.02828461,  3.74699792,  4.40152004,  4.80989846,  4.92748745,
         4.93513599]])

In [213]:
#%%timeit
fisher(theta0, grad2, data, inverse_info2, max_iter=1000)

array([[ 10.        ,  49.77371646,  49.77371646, ...,  49.77371646,
         49.77371646,  49.77371646],
       [  2.        , 202.28804783, 201.78834543, ...,   4.93516579,
          4.93516579,   4.93516579]])

In [220]:
#%%timeit
fisher(np.array([44,2])[:, None], grad2, data, inverse_info2, max_iter=30)

array([[44.        , 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646],
       [ 2.        ,  8.7114579 ,  8.37192715,  8.04567627,  7.73380208,
         7.43740659,  7.15756255,  6.89527022,  6.65140639,  6.42666857,
         6.22151895,  6.03613483,  5.87037268,  5.72375283,  5.59546926,
         5.48442526,  5.38929121,  5.30857705,  5.24070986,  5.18410739,
         5.13724021,  5.09867863,  5.06712321,  5.04142042,  5.0205662 ,
         5.00370076,  4.9900977 ,  4.97915011,  4.97035543,  4.96330059,
         4.95764809]])

In [225]:
#%%timeit
#pretty unstable
bfgs(neg_l2, theta0, neg_grad2, data)

/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


array([[ 1.00000000e+01,  3.20732160e+02,  1.10988431e+03,
         1.07689128e+03,  1.09274097e+03,  1.09274097e+03,
         1.09274097e+03,  1.09274097e+03, -1.08960272e+04,
        -7.41783081e+03,  1.08919716e+07],
       [ 2.00000000e+00,  6.26100149e+03,  9.86721009e+02,
         1.20698102e+03,  1.08211944e+03,  1.08211944e+03,
         1.08211944e+03,  1.08211944e+03, -4.15853555e+03,
        -3.47985155e+02,  5.00329071e+05]])

# Example 3 - Logistic Regression, no analytic MLE solution known

In [387]:
np.random.seed(0)
X = np.random.normal(1,1,(100,3))
X[:, 0] = 1
theta = np.random.normal(0,5,(3,1))
probs = 1/(1+np.exp(-X @ theta))
data = np.random.binomial(1,probs)

In [42]:
def l3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    return np.sum(np.log(probs)*data + (1-data)*np.log(1-probs))

def neg_l3(theta, data):
    return -l3(theta, data)
    
def grad3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    return X.T @ (data - probs)

def neg_grad3(theta, data):
    return -grad3(theta, data)

def inverse_hessian3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    D = np.diag((probs*(1-probs)).flatten())
    return -np.linalg.inv(X.T @ D @ X)

def inverse_info3(theta, data):
    return inverse_hessian3(theta, data)

In [389]:
print(theta)

[[-6.53263426]
 [ 8.2906534 ]
 [-0.59082023]]


In [390]:
theta0 = np.array([0,0,0])[:, None]

In [391]:
gd(theta0, grad3, 1/len(data), np.c_[data, X], max_iter = 10000)

array([[ 0.        ,  0.09      ,  0.02655808, ..., -4.80169901,
        -4.80169901, -4.80169901],
       [ 0.        ,  0.47420699,  0.68006613, ...,  6.45112124,
         6.45112124,  6.45112124],
       [ 0.        ,  0.06027239, -0.0236871 , ..., -0.7367028 ,
        -0.7367028 , -0.7367028 ]])

In [392]:
nr(theta0, grad3, np.c_[data, X], inverse_hessian3, max_iter=10)

array([[ 0.        , -1.32113501, -2.22772853, -3.20969671, -4.12995695,
        -4.67481577, -4.79708705, -4.80169297, -4.80169901, -4.80169901,
        -4.80169901],
       [ 0.        ,  1.55297532,  2.81474571,  4.2114976 ,  5.51644296,
         6.27718601,  6.44489008,  6.45111317,  6.45112124,  6.45112124,
         6.45112124],
       [ 0.        , -0.04207223, -0.19254868, -0.37966691, -0.58307706,
        -0.70817768, -0.73569431, -0.73670151, -0.7367028 , -0.7367028 ,
        -0.7367028 ]])

Fisher's scoring == NR for logistic regression

In [393]:
bfgs(neg_l3, theta0, neg_grad3, np.c_[data, X], max_iter = 10)
# Unstable

/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


array([[   0.        ,    9.        ,  -49.81565178,  -51.99107168,
         -59.46430646,  -72.13427956,  -50.89404287,  -54.52649807,
         -52.17169253,  -14.20036426,  158.87705477],
       [   0.        ,   47.42069856,   91.94374343,  116.27213427,
          97.51058892,   73.83804967,   81.66107679,   73.26897258,
          62.38599253,   26.78680038, -310.32436033],
       [   0.        ,    6.02723864,  -49.28597822,  -34.94956405,
          -6.6782404 ,    3.7517552 ,   -9.28580659,  -17.28250193,
          -6.10938799,   -2.93889481,  120.77775543]])

# Example 4 - Multinomial Linear Regression, higher dimensions

In [7]:
n = 5000
d = 5000
np.random.seed(0)
X = np.random.normal(0,1,(n,d))
X[:, 0] = 1
theta = np.random.normal(0,5,(d,1))
data = X @ theta + np.random.normal(0, 1, (n, 1))

In [9]:
def l4(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    return -(1/2)*np.sum((data - X @ theta)**2)

def neg_l4(theta, data):
    return -l4(theta, data)
    
def grad4(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    return -(X.T @ X @ theta - X.T @ data)

def neg_grad4(theta, data):
    return -grad4(theta, data)

def inverse_hessian4(theta, data):
    return -np.linalg.inv(X.T @ X)

def inverse_info4(theta, data):
    return inverse_hessian4(theta, data)

In [11]:
theta0 = np.zeros(d)[:, None]

In [24]:
theta

array([[-4.55078862],
       [ 2.97279141],
       [ 2.88552115],
       ...,
       [10.17179676],
       [-1.29970582],
       [-4.44035898]])

In [31]:
%%time
gd(theta0, grad4, 0.1/len(data), np.c_[data, X], max_iter = 10)

CPU times: user 30.4 s, sys: 1.25 s, total: 31.6 s
Wall time: 16.2 s


array([[ 0.        , -1.11661088, -1.83053608, ..., -2.79547374,
        -2.77211259, -2.73828815],
       [ 0.        ,  0.68471983,  1.18074619, ...,  2.36819462,
         2.43651835,  2.48956171],
       [ 0.        ,  0.26898831,  0.48450603, ...,  1.251248  ,
         1.33154855,  1.40323581],
       ...,
       [ 0.        ,  0.50710425,  0.96400212, ...,  2.81007395,
         3.01208557,  3.19427153],
       [ 0.        , -0.01158852, -0.07424399, ..., -0.66021176,
        -0.747324  , -0.82903936],
       [ 0.        , -1.51712103, -2.54898041, ..., -4.68731861,
        -4.78957973, -4.86611416]])

In [30]:
%%time
nr(theta0, grad4, np.c_[data, X], inverse_hessian4, max_iter=1)

CPU times: user 14.8 s, sys: 932 ms, total: 15.7 s
Wall time: 9.2 s


array([[ 0.        , -5.05029942],
       [ 0.        ,  2.69266304],
       [ 0.        ,  2.85097117],
       ...,
       [ 0.        , 11.07451482],
       [ 0.        , -1.70834841],
       [ 0.        , -3.26267448]])

In [16]:
%%time
bfgs(neg_l4, theta0, neg_grad4, np.c_[data, X], max_iter = 1)

1
0.5
0.25
0.125
0.0625
0.03125
0.015625
0.0078125
0.00390625
0.001953125
0.0009765625
0.00048828125
0.000244140625
0.0001220703125
CPU times: user 1min 23s, sys: 4.53 s, total: 1min 28s
Wall time: 47.9 s


array([[ 0.        , -6.81525196],
       [ 0.        ,  4.17919819],
       [ 0.        ,  1.64177436],
       ...,
       [ 0.        ,  3.0951187 ],
       [ 0.        , -0.07073068],
       [ 0.        , -9.25977192]])

# Example 5 - Real dataset

In [49]:
df = pd.read_csv("bmd.csv")
df.sex = (df.sex == "M").astype(int)
df["bmi"] = df.weight_kg / df.height_cm**2
data = (df.fracture == "fracture").astype(int).to_numpy()[:,None]
X = df[["age", "sex", "bmd", "bmi"]].to_numpy()
X = np.c_[np.ones(X.shape[0])[:,None], X]

In [58]:
theta0 = np.array([0,0,0,0,0])[:, None]
thetas = nr(theta0, grad3, np.c_[data, X], inverse_hessian3, max_iter=10)
final_theta = thetas[:,-1]
print("Accuracy: {}".format(np.mean((1/(1+np.exp(-X @ final_theta)) > 0.5) == data.flatten())))

Accuracy: 0.8579881656804734


In [ ]:
final_theta

In [66]:
thetas = gd(theta0, grad3, 1/len(data), np.c_[data, X], max_iter = 10)
final_theta = thetas[:,-1]
print("Accuracy: {}".format(np.mean((1/(1+np.exp(-X @ final_theta)) > 0.5) == data.flatten())))

Accuracy: 0.7041420118343196


/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
thetas = bfgs(neg_l3, theta0, neg_grad3, np.c_[data, X], max_iter = 10)
final_theta = thetas[:,-1]
print("Accuracy: {}".format(np.mean((1/(1+np.exp(-X @ final_theta)) > 0.5) == data.flatten())))

Accuracy: 0.4319526627218935


/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
